# Dave - Prétraitement des données

In [1]:
import pandas as pd
import os
import json

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import numpy as cp
import cupy as np

## Etape 1 - Chargement des données dans un tableau NumPy (ici CuPy)

In [2]:
def loadData():
    dfs = []
    basePath = "original_data/"
    zone_labels = os.listdir(basePath)
    for z in enumerate(zone_labels):
        df = pd.DataFrame()
        measures = os.listdir(basePath + z[1])
        for i in range(len(measures)):
            measures[i] = int(measures[i].replace(".json", ""))
        for m in sorted(measures):
            with open(basePath +  z[1] + "/" + str(m) + ".json", "r") as f:
                data = json.load(f)
                for k in data:
                    data[k] = data[k]["signalStrength"]
                    data[k] = [np.nan if v is None else v for v in data[k]]
                    data[k] = np.array(data[k], dtype=float).get()
            jDf = pd.Series(data)
            df = pd.concat([df, jDf], axis=1)
        dfs.append(df)
        dfs[-1].columns = range(30)
        
    return dfs

On en profite pour définir des paramètres importants pour notre modèle d'apprentissage, comme le nombre de salles, de réseaux, de positions mesurées et d'acquisitions par position.

In [3]:
dfs = loadData()
nb_networks = 60
nb_rooms = len(dfs)
nb_acq = 20
nb_positions = 30
nb_measures = nb_acq * nb_positions

In [4]:
values = np.zeros([nb_rooms, nb_networks, nb_positions, nb_acq], dtype="float64")
for k in range(len(dfs)):
    a = dfs[k].to_numpy()
    for i in range(len(a)):
        for j in range(len(a[i])):
            if(len(a[i][j]) > 20):
                values[k][i][j] = np.array(a[i][j][(len(a[i][j]) - nb_acq):], dtype="float64")
            else:
                values[k][i][j] = np.array(a[i][j], dtype="float64")

In [5]:
min_value = np.nanmin(values)

## Etape 2 - Analyse des données manquantes pour le choix des réseaux utilisés

In [6]:
useless_networks = []
networks_labels = []
for k in range(len(values[0])):
    mean = np.nanmean(values[:,k])
    rate = np.count_nonzero(np.isnan(values[:, k])) / (nb_rooms*nb_measures) * 100
    print(k, "%.2f" % rate, "%.2f" % mean)
    if rate > 80 and (mean < -80 or np.isnan(mean)):
        useless_networks.append(k)
    else:
        networks_labels.append(dfs[0].index[k])
        
print(useless_networks)
nb_networks -= len(useless_networks)
values = np.array(cp.delete(np.asnumpy(values), useless_networks, 1))

0 22.02 -59.55
1 7.52 -62.21
2 7.50 -62.00
3 8.67 -63.78
4 7.80 -63.96
5 13.13 -56.59
6 13.17 -56.61
7 13.18 -56.74
8 4.27 -57.76
9 41.15 -83.47
10 58.88 -81.58
11 58.52 -81.85
12 58.53 -81.81
13 26.18 -82.77
14 29.20 -82.58
15 30.82 -82.71
16 21.18 -82.79
17 47.82 -84.63
18 77.40 -81.12
19 77.55 -81.40
20 77.07 -81.36
21 22.00 -81.59
22 24.17 -81.42
23 23.30 -81.42
24 25.27 -81.40
25 87.38 -83.32
26 87.35 -82.83
27 87.62 -83.42
28 88.65 -82.87
29 83.83 -85.91
30 82.28 -86.34
31 84.40 -85.78
32 85.60 -85.62
33 44.18 -81.40
34 3.25 -63.62
35 3.33 -65.59
36 3.30 -65.65
37 3.33 -65.60
38 4.38 -55.63
39 11.50 -54.79
40 8.32 -55.09
41 3.28 -57.71
42 34.40 -82.61
43 53.98 -80.92
44 52.83 -81.12
45 54.18 -81.02
46 17.43 -83.02
47 20.45 -82.87
48 16.90 -83.48
49 100.00 nan
50 72.15 -82.63
51 76.08 -82.21
52 73.48 -82.21
53 77.32 -81.53
54 14.78 -82.18
55 27.95 -82.00
56 27.52 -81.77
57 28.93 -81.93
58 13.95 -80.10
59 80.95 -75.69
[25, 26, 27, 28, 29, 30, 31, 32, 49]


## Etape 3 - Traitement des données manquantes

Dans le cas où les données manquantes le sont en masse par position (> 80%), on les remplace par la valeur minimum mesurée sur l'entièreté des mesures.

In [7]:
for i in range(len(values)):
    for j in range(len(values[i])):
        for k in range(len(values[i][j])):
            rate = np.count_nonzero(np.isnan(values[i][j][k])) / len(values[i][j][k]) * 100
            if rate > 80:
                values[i][j][k][np.isnan(values[i][j][k])] = min_value

Dans le cas où il y a peu de données manquantes par position, on remplace les données manquantes par la moyenne des données de la position.

In [8]:
for i in range(len(values)):
    for j in range(len(values[i])):
        for k in range(len(values[i][j])):
            rate = np.count_nonzero(np.isnan(values[i][j][k])) / len(values[i][j][k]) * 100
            if rate < 30:
                values[i][j][k][np.isnan(values[i][j][k])] = int(np.nanmean(values[i][j][k]))

On enlève la dimension "position" dans la salle, les données ne sont plus triées que par salle, par réseau.

In [9]:
values = np.reshape(values, [nb_rooms, nb_networks, nb_measures])

Les dernières données manquantes sont remplacées par la moyenne des valeurs par réseau par salle.

In [10]:
for i in range(len(values)):
    for j in range(len(values[i])):
        values[i][j][np.isnan(values[i][j])] = int(np.nanmean(values[i][j]))

## Etape 4 - Traitement des données aberrantes

In [11]:
alpha = 3
total_outliers = 0
for i in range(len(values)):
    for j in range(len(values[i])):
        mean = np.mean(values[i][j])
        std = np.std(values[i][j])
        outliers = values[i][j][abs(values[i][j] - mean) > alpha * std]
        if len(outliers) > 0:
            total_outliers += len(outliers)
            values[i][j][values[i][j] > mean + alpha * std] = int(mean + alpha * std)
            values[i][j][values[i][j] < mean - alpha * std] = int(mean - alpha * std)
            
print((total_outliers / (nb_rooms * nb_networks * nb_measures)) * 100, '% of outliers replaced')

1.2673202614379087 % of outliers replaced


In [12]:
data = np.zeros((nb_networks+1, nb_measures*nb_rooms))
networks_labels.append("Labels")
for i in range(len(values)):
    data[:,i*nb_measures:(i+1)*nb_measures] = np.append(values[i], np.array([[i]] * nb_measures).T, axis=0)

In [13]:
data_df = pd.DataFrame(data.T.get())
data_df.to_csv("cleaned_data.csv", header=networks_labels, index=None)